In [9]:
#r "nuget: Moq"
#r "../../bin/Debug/net9.0/DiscordTaskBotV2.dll"

Installed Packages Moq, 4.20.72

In [10]:
using Moq;
using DiscordTaskBot.Core;
using DiscordTaskBot.Infrastructure;

const ulong TASK_CHANNEL = 123456789;

const ulong ADMIN_ID = 10000;
const ulong USER_ID = 12345;

var userInfoMock = new Mock<IUserInformationService>();
var repoMock = new Mock<ITaskRepository>();
var discordMock = new Mock<IDiscordMessageService>();

// admin user - 10000
// usernames: 10000 - Mark, 12345 - Steven

var task1 = new TaskItem("do something", new TaskDuration(DateTime.UtcNow, DateTime.UtcNow.AddDays(10), TimeZoneInfo.Utc), USER_ID); // Task for Steven, 10 days to complete
task1.SetLocation(new TaskLocation(TASK_CHANNEL, 1234));

var task2 = new TaskItem("do something else", new TaskDuration(DateTime.UtcNow, DateTime.UtcNow.AddDays(5), TimeZoneInfo.Utc), ADMIN_ID);
task2.SetLocation(new TaskLocation(TASK_CHANNEL, 2345)); // Task for Mark, 5 days to complete

// tasks:
List<TaskItem> tasks = [task1, task2];


userInfoMock
    .Setup(u => u.IsAdminAsync(It.Is<ulong>(id => id == ADMIN_ID)))
    .ReturnsAsync(true);

userInfoMock
    .Setup(u => u.IsAdminAsync(It.Is<ulong>(id => id != ADMIN_ID)))
    .ReturnsAsync(false);

userInfoMock
    .Setup(u => u.GetUsernameAsync(It.Is<ulong>(id => id == ADMIN_ID)))
    .ReturnsAsync("Mark");

userInfoMock
    .Setup(u => u.GetUsernameAsync(It.Is<ulong>(id => id == USER_ID)))
    .ReturnsAsync("Steven");

userInfoMock
    .Setup(u => u.GetUsernameAsync(It.Is<ulong>(id => id != USER_ID && id != ADMIN_ID)))
    .Throws(new InfrastructureException("User of specified id could not be found"));





repoMock
    .Setup(r => r.AddTaskAsync(It.IsAny<TaskItem>()))
    .Callback<TaskItem>(task =>
    {
        tasks.Add(task);
        Console.WriteLine($"[LOG] Task added: {task.Description} for user {task.AssigneeID}");
    })
    .Returns(Task.CompletedTask);

// DeleteTaskByIDAsync: checks if task exists, removes if found, returns true; otherwise returns false
repoMock
    .Setup(r => r.DeleteTaskByIDAsync(It.IsAny<string>()))
    .Returns<string>(taskID =>
    {
        var task = tasks.FirstOrDefault(t => t.ID == taskID);
        if (task != null)
        {
            Console.WriteLine($"[LOG] Task deleted: {task.Description}");
            tasks.Remove(task);
            return Task.FromResult(true);
        }
        Console.WriteLine($"[LOG] Could not find task of id {taskID}");
        return Task.FromResult(false);
    });

// GetTaskByIDAsync: returns the task if it exists, null if not
repoMock
    .Setup(r => r.GetTaskByIDAsync(It.IsAny<string>()))
    .Returns<string>(taskID =>
    {
        var task = tasks.FirstOrDefault(t => t.ID == taskID);
        return Task.FromResult<TaskItem?>(task);
    });

// UpdateTaskAsync: logs update if task exists, throws InfrastructureException if not
repoMock
    .Setup(r => r.UpdateTaskAsync(It.IsAny<TaskItem>()))
    .Callback<TaskItem>(task =>
    {
        var existingTask = tasks.FirstOrDefault(t => t.ID == task.ID);
        if (existingTask != null)
        {
            Console.WriteLine($"[LOG] Task updated: {task.Description}");
            // Optionally update the task in the array
            tasks = tasks.Select(t => t.ID == task.ID ? task : t).ToList();
        }
        else
        {
            throw new InfrastructureException($"Task with ID {task.ID} not found for update");
        }
    })
    .Returns(Task.CompletedTask);


// Mock CreateTaskMessageAsync
discordMock
    .Setup(d => d.CreateTaskMessageAsync(It.IsAny<ulong>(), It.IsAny<TaskItem>()))
    .ReturnsAsync((ulong channelID, TaskItem taskItem) =>
    {
        Console.WriteLine($"Creating task message: {taskItem.Description}");
        return new TaskLocation(channelID, 12345); // fixed messageID
    });

// Mock MoveTaskMessageToArchiveAsync
discordMock
    .Setup(d => d.MoveTaskMessageToArchiveAsync(It.IsAny<TaskItem>()))
    .Returns<TaskItem>(taskItem =>
    {
        if (!tasks.Contains(taskItem))
        {
            throw new InfrastructureException("TaskItem not found in tasks array");
        }

        Console.WriteLine($"Moving task '{taskItem.Description}' to archive channel");
        taskItem.SetLocation(new TaskLocation(99999, 54321)); // fixed channelID and messageID
        return Task.FromResult(taskItem.TaskLocation);
    });

// Mock DeleteTaskMessageAsync
discordMock
    .Setup(d => d.DeleteTaskMessageAsync(It.IsAny<TaskLocation>()))
    .ReturnsAsync((TaskLocation location) =>
    {
        var taskItem = tasks.FirstOrDefault(t => t.TaskLocation != null
                                                && t.TaskLocation.ChannelID == location.ChannelID
                                                && t.TaskLocation.MessageID == location.MessageID);
        if (taskItem != null)
        {
            Console.WriteLine($"Deleting task message '{taskItem.Description}' from channel {location.ChannelID}");
            return true;
        }

        return false;
    });


var taskService = new TaskService(repoMock.Object, userInfoMock.Object, discordMock.Object);

In [ ]:
// Non administrator tries to create task
try {
    await taskService.CreateTaskAsync("do something funny", new TaskDuration(DateTime.UtcNow, DateTime.UtcNow.AddDays(10), TimeZoneInfo.Utc), USER_ID, USER_ID, TASK_CHANNEL);
}
catch (Exception ex) {
    Console.WriteLine($"Error: {ex.Message}");
}

Error: Only administrators can perform that action


In [12]:
// Administrator tries to create task
await taskService.CreateTaskAsync("do something funny", new TaskDuration(DateTime.UtcNow, DateTime.UtcNow.AddDays(10), TimeZoneInfo.Utc), ADMIN_ID, USER_ID, TASK_CHANNEL);
foreach (var task in tasks) {
    Console.WriteLine(task.Description);
}

Creating task message: do something funny
[LOG] Task added: do something funny for user 12345
do something
do something else
do something funny


In [13]:
// Non administrator tries to delete task
try {
    await taskService.DeleteTaskAsync(tasks.Last().ID, USER_ID);
}
catch (Exception ex) {
    Console.WriteLine($"Error: {ex.Message}");
}

Error: Only administrators can perform that action


In [14]:
// Administrator tries to delete task
await taskService.DeleteTaskAsync(tasks.Last().ID, ADMIN_ID);

[LOG] Task deleted: do something funny


In [16]:
// Attempt to delete non existent task
try {
    await taskService.DeleteTaskAsync("nonexistenttask", ADMIN_ID);
}
catch (Exception ex) {
    Console.WriteLine($"Error: {ex.Message}");
}

[LOG] Could not find task of id nonexistenttask
Error: Task was not found
